In [1]:
# YOLO_PREDICTIONS -> [NN] -> VOT_GT

In [3]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [4]:
import os
import os.path as osp
import numpy as np

In [165]:
# GLOBALS
# vot_path = "../../../vot2016/"
vot_path = "/home/zabulskyy/Datasets/vot2016/"
# yolo_pred_path = "../yolo_predictions/extended"
yolo_pred_path = "/mnt/sshfs/YOLO-tracker/yolo_predictions/casual"
crit_vals = {
    "iou": 1,
    "dist2": 1,
#     "abs_pos": 1,
#     "rel_pos": 1,
    "cc": 100,  # class_correspondence
}
with open("../../yoloTorch/data/coco.names", "r") as f:
    classes = f.read().split("\n")[:-1]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [166]:
def read_full_gt(vot_path=vot_path, force_square=True, return_format="dict"):
    vot_classes = [x for x in os.listdir(vot_path) if not x.endswith(".txt")]
    if return_format == "list":
        X = list()
    else:
        X = dict()
    for vot_class in vot_classes:
        gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
        with open(gt_file, 'r') as file:
            l =file.read().split("\n")
            l.pop(-1)
            l = [[float(x) for x in y.split(",")] for y in l]
            if force_square:
                l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]   
        
        if return_format == "list":
            X.append(l)
        else:
            X[vot_class] = l
                    
    return X

In [168]:
def read_all_yolo_preds(path=yolo_pred_path, pred_format="csv", return_format="dict", threshold=.9):
    if return_format == "list":
        X = list()
    else:
        X = dict()
    
    for vot_class in os.listdir(path):
        yolo_pred_file = osp.join(path, vot_class)
        with open(yolo_pred_file, 'r') as file:
            l = file.read().split("\n")
            l.pop(-1)
            l.pop(0)
            im_len = -1
            
            l = np.array([np.array([float(x) for x in i.split(",")]) for i in l])
            l = np.array([x[[0,1,2,3,4,7]] for x in l])# if x[6] > threshold])
            
            X[vot_class[:-4]] = l
    return X

In [169]:
X = read_all_yolo_preds()


In [170]:
X["birds2"][X["birds2"][:, 0] == 0][:].astype(float)

array([[  0.        , 634.37915039, 244.24494934, 736.13616943,
        487.36264038,   0.        ],
       [  0.        , 694.62506104, 214.65588379, 843.2197876 ,
        459.17602539,   0.        ],
       [  0.        ,  75.28208923, 387.6053772 , 154.14573669,
        499.74859619,  14.        ],
       [  0.        , 208.15631104, 368.37866211, 322.29425049,
        488.35415649,  14.        ],
       [  0.        , 296.33383179, 370.64733887, 406.98562622,
        479.33563232,  14.        ],
       [  0.        , 435.77996826, 384.60476685, 536.97509766,
        460.79129028,  14.        ]])

In [129]:
def read_full_gt(path=vot_path, pred_format="csv", return_format="dict"):
    if return_format == "list":
        X = list()
    else:
        X = dict()
    
    for vot_class in [x for x in os.listdir(path) if not x.endswith(".txt")]:
        yolo_pred_file = osp.join(path, vot_class, "groundtruth.txt")
        with open(yolo_pred_file, 'r') as file:
            l = file.read().split("\n")
            l.pop(-1)
            l.pop(0)
            im_len = -1
            l = np.array([[float(x) for x in i.split(",")] for i in l])
            l = np.array([[n, min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for n, m in enumerate(l)])
            X[vot_class] = l
    return X

In [130]:
y = read_full_gt()

In [171]:
y["birds2"][0], X["birds2"][X["birds2"][:, 0] == 0].astype(int)

(array([  0.  , 441.66, 372.33, 528.22, 451.47]),
 array([[  0, 634, 244, 736, 487,   0],
        [  0, 694, 214, 843, 459,   0],
        [  0,  75, 387, 154, 499,  14],
        [  0, 208, 368, 322, 488,  14],
        [  0, 296, 370, 406, 479,  14],
        [  0, 435, 384, 536, 460,  14]]))

In [46]:
def read_csv(file):
    with open(file, 'r') as file:
        l = file.read().split("\n")
        l.pop(-1)
        l = [[float(x) for x in y.split(",")[:-1]] for y in l]   
        return torch.tensor(l)
    

In [57]:
birds2 = X["birds2"]
type(birds2)

list

In [16]:
X["blanket"]

[224.0,
 137.0874481201172,
 121.8482437133789,
 161.82241821289062,
 136.82615661621094,
 0.0011430482845753431,
 0.15401728451251984,
 77.0]

In [176]:
import torch

In [177]:
D_in, H, D_out = 6, 100, 5

In [178]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)